In [1]:
import pandas as pd
import json
from collections import defaultdict
import csv
from argparse import ArgumentParser
import numpy as np
import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences
from keras.utils import Progbar
from random import randint

from scipy import stats
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

import itertools

from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import inflect

%matplotlib inline

import sys

sys.path.append('/home/ece/Desktop/nlp/vqa/FOIL-resources/coco/PythonAPI')
sys.path.append('/home/ece/Desktop/nlp/vqa/FOIL-resources/coco/PythonAPI/pycocotools/')

from pycocotools.coco import COCO

import json
from pprint import pprint
import re
import nltk
import copy
import random
import math
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import pickle
import os.path

from mosestokenizer import *
tokenize = MosesTokenizer('en')

Using TensorFlow backend.


In [2]:
preprocessed_foil_train = pd.read_csv(
    "foil_train.tsv",
    sep="\t",
    names=["label", "caption", "jpg"]
)

foil_words = set()
with open("foilv1.0_train2017.json") as in_file:
    foil_train = json.load(in_file)
    for annotation in foil_train["annotations"]:
        foil_words.add(annotation["foil_word"])
        
        

In [3]:
foil_train

{'annotations': [{'caption': 'A long restaurant table with rattan rounded back chairs.',
   'foil_id': 2000001,
   'foil_word': 'ORIG',
   'id': 789366,
   'image_id': 57870,
   'target_word': 'ORIG'},
  {'caption': 'a long table with a plant on top of it surrounded with wooden chairs ',
   'foil_id': 2000002,
   'foil_word': 'ORIG',
   'id': 789888,
   'image_id': 57870,
   'target_word': 'ORIG'},
  {'caption': 'A restaurant has modern wooden tables and chairs.',
   'foil_id': 2000004,
   'foil_word': 'ORIG',
   'id': 787980,
   'image_id': 57870,
   'target_word': 'ORIG'},
  {'caption': 'a chair is adorned with wooden chairs with blue accents.',
   'foil_id': 2000005,
   'foil_word': 'chair',
   'id': 794853,
   'image_id': 57870,
   'target_word': 'table'},
  {'caption': 'a long toilet with a flower arrangement in the middle for meetings',
   'foil_id': 2000006,
   'foil_word': 'toilet',
   'id': 791316,
   'image_id': 57870,
   'target_word': 'table'},
  {'caption': 'a person stand

In [4]:
with open("cocoObject_target.txt") as in_file:
    obj_targ = [l.split()[0] for l in in_file.readlines()]
    
print(obj_targ)

['airplane', 'apple', 'backpack', 'ball', 'banana', 'bat', 'bear', 'bed', 'bench', 'bicycle', 'bird', 'boat', 'book', 'bottle', 'bowl', 'broccoli', 'bus', 'cake', 'car', 'carrot', 'cat', 'chair', 'clock', 'couch', 'cow', 'cup', 'dog', 'donut', 'drier', 'elephant', 'fork', 'frisbee', 'giraffe', 'glass', 'glove', 'handbag', 'horse', 'hydrant', 'keyboard', 'kite', 'knife', 'laptop', 'microwave', 'motorcycle', 'mouse', 'orange', 'oven', 'phone', 'pizza', 'racket', 'refrigerator', 'remote', 'sandwich', 'scissors', 'sheep', 'sink', 'skateboard', 'skis', 'snowboard', 'spoon', 'suitcase', 'surfboard', 'table', 'tie', 'toaster', 'toilet', 'toothbrush', 'train', 'truck', 'tv', 'umbrella', 'vase', 'zebra']


In [5]:
with open("annotations_trainval/captions_train2017.json") as in_file:
    train_coco = json.load(in_file)

In [6]:
train_captions_per_image = dict()

In [7]:
print(train_coco['annotations'][0])

{'image_id': 203564, 'id': 37, 'caption': 'A bicycle replica with a clock as the front wheel.'}


In [8]:
for tc in train_coco['annotations']:
    if tc['image_id'] in train_captions_per_image:
        train_captions_per_image[tc['image_id']].append((tc['caption'], tc['id']))
    else:
        train_captions_per_image[tc['image_id']] = [(tc['caption'], tc['id'])]
        

In [9]:
train_captions_per_image

{203564: [('A bicycle replica with a clock as the front wheel.', 37),
  ('The bike has a clock as a tire.', 181),
  ('A black metal bicycle with a clock inside the front wheel.', 478),
  ('A bicycle figurine in which the front wheel is replaced with a clock\n',
   6637),
  ('A clock with the appearance of the wheel of a bicycle ', 6802)],
 322141: [('A room with blue walls and a white sink and door.', 49),
  ('Blue and white color scheme in a small bathroom.', 109),
  ('This is a blue and white bathroom with a wall sink and a lifesaver on the wall.',
   121),
  ('A blue boat themed bathroom with a life preserver on the wall', 163),
  ('A bathroom with walls that are painted baby blue.', 250)],
 16977: [('A car that seems to be parked illegally behind a legally parked car',
   89),
  ('two cars parked on the sidewalk on the street', 185),
  ('City street with parked cars and a bench.', 626),
  ('Cars try to maneuver into parking spaces along a densely packed city street. ',
   1229),
  

In [10]:
with open("annotations_trainval/captions_val2017.json") as in_file:
    val_coco = json.load(in_file)
        
val_captions_per_image = dict()

for vc in val_coco['annotations']:
    if vc['image_id'] in val_captions_per_image:
        val_captions_per_image[vc['image_id']].append((vc['caption'], vc['id']))
    else:
        val_captions_per_image[vc['image_id']] = [(vc['caption'], vc['id'])]

In [11]:
with open("annotations_test/image_info_test2017.json") as in_file:
    test_cap = json.load(in_file)

In [12]:
test_cap['images'][0]

{'coco_url': 'http://images.cocodataset.org/test2017/000000466319.jpg',
 'date_captured': '2013-11-14 11:04:33',
 'file_name': '000000466319.jpg',
 'height': 480,
 'id': 466319,
 'license': 6,
 'width': 640}

In [13]:
test_cap['categories'] #object categories

[{'id': 1, 'name': 'person', 'supercategory': 'person'},
 {'id': 2, 'name': 'bicycle', 'supercategory': 'vehicle'},
 {'id': 3, 'name': 'car', 'supercategory': 'vehicle'},
 {'id': 4, 'name': 'motorcycle', 'supercategory': 'vehicle'},
 {'id': 5, 'name': 'airplane', 'supercategory': 'vehicle'},
 {'id': 6, 'name': 'bus', 'supercategory': 'vehicle'},
 {'id': 7, 'name': 'train', 'supercategory': 'vehicle'},
 {'id': 8, 'name': 'truck', 'supercategory': 'vehicle'},
 {'id': 9, 'name': 'boat', 'supercategory': 'vehicle'},
 {'id': 10, 'name': 'traffic light', 'supercategory': 'outdoor'},
 {'id': 11, 'name': 'fire hydrant', 'supercategory': 'outdoor'},
 {'id': 13, 'name': 'stop sign', 'supercategory': 'outdoor'},
 {'id': 14, 'name': 'parking meter', 'supercategory': 'outdoor'},
 {'id': 15, 'name': 'bench', 'supercategory': 'outdoor'},
 {'id': 16, 'name': 'bird', 'supercategory': 'animal'},
 {'id': 17, 'name': 'cat', 'supercategory': 'animal'},
 {'id': 18, 'name': 'dog', 'supercategory': 'animal'},

In [14]:
with open("annotations_test/image_info_test2014.json") as in_file:
    test_cap2014 = json.load(in_file)
    
print(len(test_cap2014['categories']))

for c in range(len(test_cap2014['categories'])):
    print(test_cap['categories'][c]['name'], test_cap2014['categories'][c]['name'], )

80
person person
bicycle bicycle
car car
motorcycle motorcycle
airplane airplane
bus bus
train train
truck truck
boat boat
traffic light traffic light
fire hydrant fire hydrant
stop sign stop sign
parking meter parking meter
bench bench
bird bird
cat cat
dog dog
horse horse
sheep sheep
cow cow
elephant elephant
bear bear
zebra zebra
giraffe giraffe
backpack backpack
umbrella umbrella
handbag handbag
tie tie
suitcase suitcase
frisbee frisbee
skis skis
snowboard snowboard
sports ball sports ball
kite kite
baseball bat baseball bat
baseball glove baseball glove
skateboard skateboard
surfboard surfboard
tennis racket tennis racket
bottle bottle
wine glass wine glass
cup cup
fork fork
knife knife
spoon spoon
bowl bowl
banana banana
apple apple
sandwich sandwich
orange orange
broccoli broccoli
carrot carrot
hot dog hot dog
pizza pizza
donut donut
cake cake
chair chair
couch couch
potted plant potted plant
bed bed
dining table dining table
toilet toilet
tv tv
laptop laptop
mouse mouse
remote 

In [15]:
obj_cat = []
super_cat = []
obj_sup_cat = dict()
mult_obj_cat = []
mult_sup_cat = dict()

for c in test_cap['categories']:
    if len(c['name'].split()) == 1:
        obj_cat.append(c['name']) 
        super_cat.append(c['supercategory'])
        
        if c['supercategory'] in obj_sup_cat:
            obj_sup_cat[c['supercategory']].append(c['name'])
        else:
            obj_sup_cat[c['supercategory']] = [c['name']]
            
    else:
        mult_obj_cat.append(c['name'])
        
        if c['supercategory'] in mult_sup_cat:
            mult_sup_cat[c['supercategory']].append(c['name'])
        else:
            mult_sup_cat[c['supercategory']] = [c['name']]
        
        

In [16]:
# obj_cat = sorted(obj_cat)
# #print(len(obj_cat)
# print(obj_cat)
# print()
# print(len(mult_obj_cat), mult_obj_cat)

all_cat = []
all_cat.extend(obj_cat)
all_cat.extend(mult_obj_cat)
all_cat = sorted(all_cat)

In [17]:
len(all_cat)

80

In [18]:
#different compared to the file I received from Ravi 
#categories such as: dining table vs. table, sports ball vs. ball

print(len(obj_targ), obj_targ)

73 ['airplane', 'apple', 'backpack', 'ball', 'banana', 'bat', 'bear', 'bed', 'bench', 'bicycle', 'bird', 'boat', 'book', 'bottle', 'bowl', 'broccoli', 'bus', 'cake', 'car', 'carrot', 'cat', 'chair', 'clock', 'couch', 'cow', 'cup', 'dog', 'donut', 'drier', 'elephant', 'fork', 'frisbee', 'giraffe', 'glass', 'glove', 'handbag', 'horse', 'hydrant', 'keyboard', 'kite', 'knife', 'laptop', 'microwave', 'motorcycle', 'mouse', 'orange', 'oven', 'phone', 'pizza', 'racket', 'refrigerator', 'remote', 'sandwich', 'scissors', 'sheep', 'sink', 'skateboard', 'skis', 'snowboard', 'spoon', 'suitcase', 'surfboard', 'table', 'tie', 'toaster', 'toilet', 'toothbrush', 'train', 'truck', 'tv', 'umbrella', 'vase', 'zebra']


In [19]:
#mismatches

# R ball
# R bat
# R drier
# R glass
# R glove
# R hydrant
# R phone
# R racket
# R table


# E 'baseball bat' bat
# E 'baseball glove' glove
# E 'cell phone' phone
# E 'dining table' table
# E 'fire hydrant' hydrant
# E 'hair drier' drier
# E 'hot dog' dog
# E 'parking meter' NOT IN OLD LIST
# E 'person' NOT IN OLD LIST  +person
# E 'potted plant' NOT IN OLD LIST +plant
# E 'sports ball' ball
# E 'stop sign' NOT IN OLD LIST +sign
# E 'teddy bear' NOT IN OLD LIST
# E 'tennis racket'racket
# E 'traffic light' NOT IN OLD LIST +light
# E 'wine glass' glass

In [20]:
# obj_targ.extend(['person', 'plant', 'sign', 'light'])
# obj_targ = sorted(obj_targ)

with open('new_obj_categories.txt', 'w') as f:
    
    for c in range(len(obj_targ)):
        f.write(obj_targ[c])    
        if c < (len(obj_targ) - 1):
            f.write('\n')

In [21]:
with open('new_obj_categories.txt', 'r') as f:
    foil2_categories = [l.split()[0] for l in f.readlines()]

In [22]:
len(foil2_categories)

73

In [23]:
#manual addition of single-word categories, obtained from multi-word counterparts

obj_sup_cat['outdoor'].append('light')
obj_sup_cat['outdoor'].append('hydrant')
obj_sup_cat['outdoor'].append('sign')

obj_sup_cat['sports'].append('ball')
obj_sup_cat['sports'].append('bat')
obj_sup_cat['sports'].append('glove')
obj_sup_cat['sports'].append('racket')

obj_sup_cat['kitchen'].append('glass')

obj_sup_cat['furniture'].append('plant')
obj_sup_cat['furniture'].append('table')

obj_sup_cat['electronic'].append('phone')

obj_sup_cat['indoor'].append('drier')


In [24]:
count = 0

for c in obj_sup_cat:
    count += len(obj_sup_cat[c])
    
count

77

In [25]:
target_foil_pairs = []
train_pairs = []
test_pairs = []

for cat in obj_sup_cat:
    if len(obj_sup_cat[cat]) > 1: #there are some supercategories with 1 object category
        perms = list(itertools.permutations(obj_sup_cat[cat], 2))
        target_foil_pairs.extend(perms)
        
        for p in perms:
            if randint(0, 1) == 1:
                train_pairs.append(p)
            else:
                test_pairs.append(p)

print(target_foil_pairs)

[('bicycle', 'car'), ('bicycle', 'motorcycle'), ('bicycle', 'airplane'), ('bicycle', 'bus'), ('bicycle', 'train'), ('bicycle', 'truck'), ('bicycle', 'boat'), ('car', 'bicycle'), ('car', 'motorcycle'), ('car', 'airplane'), ('car', 'bus'), ('car', 'train'), ('car', 'truck'), ('car', 'boat'), ('motorcycle', 'bicycle'), ('motorcycle', 'car'), ('motorcycle', 'airplane'), ('motorcycle', 'bus'), ('motorcycle', 'train'), ('motorcycle', 'truck'), ('motorcycle', 'boat'), ('airplane', 'bicycle'), ('airplane', 'car'), ('airplane', 'motorcycle'), ('airplane', 'bus'), ('airplane', 'train'), ('airplane', 'truck'), ('airplane', 'boat'), ('bus', 'bicycle'), ('bus', 'car'), ('bus', 'motorcycle'), ('bus', 'airplane'), ('bus', 'train'), ('bus', 'truck'), ('bus', 'boat'), ('train', 'bicycle'), ('train', 'car'), ('train', 'motorcycle'), ('train', 'airplane'), ('train', 'bus'), ('train', 'truck'), ('train', 'boat'), ('truck', 'bicycle'), ('truck', 'car'), ('truck', 'motorcycle'), ('truck', 'airplane'), ('tru

In [26]:
len(target_foil_pairs)

492

In [27]:
test_pairs

[('bicycle', 'motorcycle'),
 ('bicycle', 'bus'),
 ('bicycle', 'boat'),
 ('car', 'motorcycle'),
 ('car', 'airplane'),
 ('car', 'train'),
 ('car', 'truck'),
 ('car', 'boat'),
 ('motorcycle', 'car'),
 ('motorcycle', 'airplane'),
 ('motorcycle', 'bus'),
 ('motorcycle', 'truck'),
 ('airplane', 'bicycle'),
 ('airplane', 'car'),
 ('airplane', 'bus'),
 ('airplane', 'train'),
 ('bus', 'car'),
 ('train', 'car'),
 ('train', 'motorcycle'),
 ('train', 'airplane'),
 ('train', 'bus'),
 ('train', 'boat'),
 ('truck', 'bicycle'),
 ('truck', 'airplane'),
 ('truck', 'bus'),
 ('truck', 'train'),
 ('boat', 'bicycle'),
 ('boat', 'car'),
 ('boat', 'motorcycle'),
 ('boat', 'airplane'),
 ('bench', 'sign'),
 ('light', 'bench'),
 ('light', 'sign'),
 ('hydrant', 'light'),
 ('hydrant', 'sign'),
 ('sign', 'bench'),
 ('sign', 'light'),
 ('sign', 'hydrant'),
 ('bird', 'cow'),
 ('bird', 'elephant'),
 ('bird', 'bear'),
 ('bird', 'zebra'),
 ('cat', 'bird'),
 ('cat', 'dog'),
 ('cat', 'horse'),
 ('cat', 'cow'),
 ('cat', 'e

In [28]:
len(test_pairs)

241

In [29]:
train_pairs

[('bicycle', 'car'),
 ('bicycle', 'airplane'),
 ('bicycle', 'train'),
 ('bicycle', 'truck'),
 ('car', 'bicycle'),
 ('car', 'bus'),
 ('motorcycle', 'bicycle'),
 ('motorcycle', 'train'),
 ('motorcycle', 'boat'),
 ('airplane', 'motorcycle'),
 ('airplane', 'truck'),
 ('airplane', 'boat'),
 ('bus', 'bicycle'),
 ('bus', 'motorcycle'),
 ('bus', 'airplane'),
 ('bus', 'train'),
 ('bus', 'truck'),
 ('bus', 'boat'),
 ('train', 'bicycle'),
 ('train', 'truck'),
 ('truck', 'car'),
 ('truck', 'motorcycle'),
 ('truck', 'boat'),
 ('boat', 'bus'),
 ('boat', 'train'),
 ('boat', 'truck'),
 ('bench', 'light'),
 ('bench', 'hydrant'),
 ('light', 'hydrant'),
 ('hydrant', 'bench'),
 ('bird', 'cat'),
 ('bird', 'dog'),
 ('bird', 'horse'),
 ('bird', 'sheep'),
 ('bird', 'giraffe'),
 ('cat', 'sheep'),
 ('cat', 'bear'),
 ('cat', 'zebra'),
 ('dog', 'bird'),
 ('dog', 'cat'),
 ('dog', 'horse'),
 ('dog', 'cow'),
 ('dog', 'zebra'),
 ('horse', 'cat'),
 ('horse', 'sheep'),
 ('horse', 'cow'),
 ('horse', 'elephant'),
 ('hors

In [30]:
len(train_pairs)

251

In [31]:
def convert_sentence(orig_sent, orig_word, foil_word):
    #change only the tokens, not accidental ones etc.
    
    orig_list = orig_sent.split()
    
    for o in range(len(orig_list)):
        
        if orig_list[o] == orig_word:
            orig_list[o] = foil_word
            
            
    foil_sent = ''
    for o in range(len(orig_list)):
        foil_sent = foil_sent.join(orig_list)
        
        if o < len(orig_list)-1:
            foil_sent = foil_sent.join(' ')
        
    return foil_sent


def find_object_mentions(captions, obj_cat, lem, constraint_mentions):
    
    freq_words = defaultdict(int)
    
    #find the visually salient object words, ones that were mentioned multiple times in the captions
    for c in obj_cat:
        for cap in captions:
            sent = cap[0]
            for w in sent.split():
                if w.lower() == c or lem.lemmatize(w.lower()) == c: #convert to lowercase and lemmatize            
                    freq_words[c] += 1
    
    #todo: at least 2 user mentioned the word
    temp_freq_words = freq_words.copy()
    
    for f in temp_freq_words:
        if freq_words[f] < constraint_mentions:
            freq_words.pop(f)
            
    return freq_words


def generate_not_present_foil(captions, img_id, object_instances, salient_words, obj_cat, target_foil_pairs, lem):
    
    foil_pairs = []
    
    word_mode = defaultdict(list)
    #find potential foil words from the same supercategory as the salient word
    for sw in salient_words:
            for tf in target_foil_pairs:
                if tf[0] == sw:
                    potential_foil = tf[1]
                    foil_pairs.append((sw, potential_foil))
                   
    #remove the foil words that were mentioned in the captions
    
    #NEW! also check the instances set for the object annotations
    #remove object categories annotated as being present in the image
    
    obj_insts = object_instances[img_id]
    
    temp_foil_pairs = foil_pairs
    
    for fp in temp_foil_pairs:
        for c in captions:
            sent = c[0]
            for w in sent.split():
                if w.lower() == fp[1] or lem.lemmatize(w) == fp[1]:  
                    if fp in foil_pairs:
                        foil_pairs.remove(fp)
              
        for obj in obj_insts:
            if obj == fp[1]: #obj is the same as foil word
                if fp in foil_pairs:
                    foil_pairs.remove(fp)
    
    return foil_pairs

In [32]:
lemmatizer = WordNetLemmatizer()
word = 'mice'
print(lemmatizer.lemmatize(word))

p = inflect.engine()
print(p.plural('Mouse'))
plurals = defaultdict(str)

for obj in obj_cat:
    plurals[obj] = p.plural(obj)

mouse
Mice


In [33]:
constraint_mentions = 2 #at least 2 caption includes that category

In [48]:
for tc in train_captions_per_image[322141]:
    print(tc)
salient_words = find_object_mentions(train_captions_per_image[322141], obj_cat, lemmatizer, constraint_mentions)
print()
print(salient_words)
foil_pairs = generate_not_present_foil(train_captions_per_image[203564], 203564, train_object_instances, salient_words, obj_cat, target_foil_pairs, lemmatizer)
print()
print(foil_pairs)

('A room with blue walls and a white sink and door.', 49)
('Blue and white color scheme in a small bathroom.', 109)
('This is a blue and white bathroom with a wall sink and a lifesaver on the wall.', 121)
('A blue boat themed bathroom with a life preserver on the wall', 163)
('A bathroom with walls that are painted baby blue.', 250)

defaultdict(<class 'int'>, {'sink': 2})

[('sink', 'microwave'), ('sink', 'oven'), ('sink', 'toaster'), ('sink', 'refrigerator')]


In [49]:
with MosesTokenizer('en') as tokenize:
    tokenize('Hello World!')

In [50]:
def get_tokenized_captions(captions):
    
    tokenized_captions = []
    
    for caption in captions:
        with MosesTokenizer('en') as tokenize:
            sentence = tokenize(caption[0])
            
        #now punctuation marks are all separated
        #combine the tokens with a space between them
        new_sentence = ''
        for t in range(len(sentence)):
            new_sentence += sentence[t]
            
            if t < (len(sentence)-1):
                new_sentence += ' '
                
        temp_caption = (new_sentence, caption[1])#tokenized_caption, sentence_id
        
        tokenized_captions.append(temp_caption)
    
    return tokenized_captions

In [51]:
def get_tokenized_sentence(sentence):
    
    tokenized_sentence = []
    
    with MosesTokenizer('en') as tokenize:
        sentence = tokenize(caption[0])
            
        #now punctuation marks are all separated
        #combine the tokens with a space between them
        new_sentence = ''
        for t in range(len(sentence)):
            new_sentence += sentence[t]
            
            if t < (len(sentence)-1):
                new_sentence += ' '
                
       
    return new_sentence

In [52]:
foil_sentences= defaultdict(list)

#for each image
for tp in train_captions_per_image:
    
    print(tp)
    #all 5 captions of an image
    captions = train_captions_per_image[tp]
    
    captions = get_tokenized_captions(captions)
    
    #find the visually salient words given all captions for an image
    vis_sal_words = find_object_mentions(captions, obj_cat, lemmatizer,constraint_mentions)
    
    #find possible foil words for the captions
    
    #IMPORTANT CHANGE: target_foil_pairs to train_pairs
    #also removing visually present objects
    foil_pairs =  generate_not_present_foil(captions, tp, train_object_instances, vis_sal_words, obj_cat, test_pairs, lemmatizer)
     
    for caption in captions:
        
        sent = caption[0]
        sentid = caption[1]
        
        for w in sent.split():

            #if the word is a salient word
            if w in vis_sal_words:
                
                for fp in foil_pairs:
                    
                    #check if that word is an original word in the list of target-foil pairs
                    #convert the sentence into foil sentence 
                    #check for singular-plural form and upper,lower,title cases
                    
                    if w == fp[0]:
                        foil_sentences[tp].append([convert_sentence(sent, w, fp[1]), w , fp[1], sentid])
                        
                    elif w == fp[0].upper():
                        foil_sentences[tp].append([convert_sentence(sent, w, fp[1].upper()), w , fp[1].upper(), sentid])
                        
                    elif w == fp[0].title():    
                        foil_sentences[tp].append([convert_sentence(sent, w, fp[1].title()), w , fp[1].title(), sentid])
                        
                    elif p.singular_noun(w) == fp[0]:
                        foil_sentences[tp].append([convert_sentence(sent, w, p.plural(fp[1])), w , p.plural(fp[1]), sentid])
                        
                    elif p.singular_noun(w) == fp[0].upper():
                        foil_sentences[tp].append([convert_sentence(sent, w, p.plural(fp[1]).upper()), w , p.plural(fp[1]).upper(), sentid])
                        
                    elif p.singular_noun(w) == fp[0].title():
                        foil_sentences[tp].append([convert_sentence(sent, w, p.plural(fp[1]).title()), w , p.plural(fp[1]).title(), sentid])



203564
322141
16977
106140
571635
301837
315702
189634


KeyboardInterrupt: 

In [ ]:
foil_sentences[203564] #foil sentence, orig word, foil word

In [ ]:
with open('train_foil_sentences_ece.json', 'w') as f:
    json.dump(foil_sentences, f)

In [ ]:
foil_sentences_val= defaultdict(list)

#for each image
for tp in val_captions_per_image:
    
    #all 5 captions of an image
    captions = val_captions_per_image[tp]
    
    captions = get_tokenized_captions(captions)
        
    #find the visually salient words given all captions for an image
    vis_sal_words = find_object_mentions(captions, obj_cat, lemmatizer,constraint_mentions)
    
    #find possible foil words for the captions
    
    #IMPORTANT CHANGE: target_foil_pairs to test_pairs
    foil_pairs =  generate_not_present_foil(captions, tp, val_object_instances, vis_sal_words, obj_cat, test_pairs, lemmatizer)
     
    for caption in captions:
        
        sent = caption[0]
        sentid = caption[1]
        
        for w in sent.split():
            
             #if the word is a salient word
            if w in vis_sal_words:
                
                for fp in foil_pairs:
                    
                    #check if that word is an original word in the list of target-foil pairs
                    #convert the sentence into foil sentence 
                    #check for singular-plural form and upper,lower,title cases
                    
                    if w == fp[0]:
                        foil_sentences_val[tp].append([convert_sentence(sent, w, fp[1]), w , fp[1], sentid])
                        
                    elif w == fp[0].upper():
                        foil_sentences_val[tp].append([convert_sentence(sent, w, fp[1].upper()), w , fp[1].upper(), sentid])
                        
                    elif w == fp[0].title():    
                        foil_sentences_val[tp].append([convert_sentence(sent, w, fp[1].title()), w , fp[1].title(), sentid])
                        
                    elif p.singular_noun(w) == fp[0]:
                        foil_sentences_val[tp].append([convert_sentence(sent, w, p.plural(fp[1])), w , p.plural(fp[1]), sentid])
                        
                    elif p.singular_noun(w) == fp[0].upper():
                        foil_sentences_val[tp].append([convert_sentence(sent, w, p.plural(fp[1]).upper()), w , p.plural(fp[1]).upper(), sentid])
                        
                    elif p.singular_noun(w) == fp[0].title():
                        foil_sentences_val[tp].append([convert_sentence(sent, w, p.plural(fp[1]).title()), w , p.plural(fp[1]).title(), sentid])



In [ ]:
foil_sentences_val[179765]

In [ ]:
val_captions_per_image[179765]

In [ ]:
with open('val_foil_sentences_ece.json', 'w') as f:
    json.dump(foil_sentences_val, f)

In [75]:
def coco_lookup(coco):
    
    lookup_table = defaultdict(dict)
    
    for im in coco['images']:
        
        cocoid = im['cocoid']
        
        filename = im['filename']
        filepath = im['filepath']
        
        imgid = im['imgid']
        
        sentids = im['sentids']
        
        sentences = im['sentences']
        
        lookup_table[cocoid] = {'filename' : filename, 'filepath':filepath, 'imgid': imgid, 'sentids':sentids, 'sentences':sentences}
    
    return lookup_table


with open("neuraltalk-master/data/coco/coco_dataset.json") as in_file:
    coco = json.load(in_file)
    

In [76]:
lookup = coco_lookup(coco)
print(lookup[203564]['filename'])
print(coco.keys())
print(coco['images'][0])

COCO_val2014_000000203564.jpg
dict_keys(['images', 'dataset'])
{'filepath': 'val2014', 'sentids': [770337, 771687, 772707, 776154, 781998], 'filename': 'COCO_val2014_000000391895.jpg', 'imgid': 0, 'split': 'test', 'sentences': [{'tokens': ['a', 'man', 'with', 'a', 'red', 'helmet', 'on', 'a', 'small', 'moped', 'on', 'a', 'dirt', 'road'], 'raw': 'A man with a red helmet on a small moped on a dirt road. ', 'imgid': 0, 'sentid': 770337}, {'tokens': ['man', 'riding', 'a', 'motor', 'bike', 'on', 'a', 'dirt', 'road', 'on', 'the', 'countryside'], 'raw': 'Man riding a motor bike on a dirt road on the countryside.', 'imgid': 0, 'sentid': 771687}, {'tokens': ['a', 'man', 'riding', 'on', 'the', 'back', 'of', 'a', 'motorcycle'], 'raw': 'A man riding on the back of a motorcycle.', 'imgid': 0, 'sentid': 772707}, {'tokens': ['a', 'dirt', 'path', 'with', 'a', 'young', 'person', 'on', 'a', 'motor', 'bike', 'rests', 'to', 'the', 'foreground', 'of', 'a', 'verdant', 'area', 'with', 'a', 'bridge', 'and', 'a

In [ ]:
with open('train_foil_sentences_ece.json', 'r') as f:
    foil_sentences = json.load(f)
    
with open('val_foil_sentences_ece.json', 'r') as f:
    foil_sentences_val = json.load(f)

In [ ]:
train_check_dataset = defaultdict(list)
train_check_dataset['dataset'] = 'train_check'

#create the dataset so that it can be utilized in NeuralTalk

for ft in foil_sentences:
    
    sentences = foil_sentences[ft]
    
    cocoid = int(ft)
    
    split = 'train'

    filename = lookup[cocoid]['filename']
    filepath = lookup[cocoid]['filepath']
    imgid = lookup[cocoid]['imgid']
    posTag = 'Original' 

    #first the original captions for this cocoid
    
    for org_cap in train_captions_per_image[cocoid]:
        
        sent = org_cap[0]
        
        sent = get_tokenized_sentence(sent)
        
        sentid = org_cap[1]
        
        #print(sent)
        changeid = 'Original'

        sentids = [sentid]

        targetfoilPair = 'Original'
        
        im_dict = defaultdict()
        
        im_dict['change_id'] = changeid
        im_dict['cocoid'] = cocoid
        im_dict['filename'] = filename
        im_dict['filepath'] = filepath
        im_dict['imgid'] = imgid
        im_dict['posTag'] = posTag
        im_dict['sentids'] = sentids
        im_dict['split'] = split
        im_dict['targetfoilPair'] = targetfoilPair


        im_sent_dict = defaultdict()
        
        im_sent_dict['imgid'] = imgid
        im_sent_dict['raw'] = sent
        im_sent_dict['sentid'] = sentid
        im_sent_dict['tokens'] = sent.split()

        im_dict['sentences'] = [im_sent_dict]
      
        train_check_dataset['images'].append(im_dict)

    c = 1 #foil counter per caption
    
    #then all the foil captions
    for s in sentences:
    
        #print(s)
        foil_sent = s[0]
        pair0 = s[1]
        pair1 = s[2]
        sentid = s[3]

        targetfoilPair = pair0 + '_' + pair1
        
        changeid = str(sentid) + '_' + str(c)
        c += 1
        sentids = [sentid]
        
        posTag = 'noun'
        
        im_dict = defaultdict()
        
        im_dict['change_id'] = changeid
        im_dict['cocoid'] = cocoid
        im_dict['filename'] = filename
        im_dict['filepath'] = filepath
        im_dict['imgid'] = imgid
        im_dict['posTag'] = posTag
        im_dict['sentids'] = sentids
        im_dict['split'] = split
        im_dict['targetfoilPair'] = targetfoilPair


        im_sent_dict = defaultdict()
        
        im_sent_dict['imgid'] = imgid
        im_sent_dict['raw'] = foil_sent
        im_sent_dict['sentid'] = sentid
        im_sent_dict['tokens'] = foil_sent.split()

        im_dict['sentences'] = [im_sent_dict]
      
        train_check_dataset['images'].append(im_dict)

In [ ]:
train_check_dataset['images'][5]['sentences']

In [ ]:
test_check_dataset = defaultdict(list)
test_check_dataset['dataset'] = 'test_check'

for ft in foil_sentences_val:
    
    sentences = foil_sentences_val[ft]
    
    cocoid = int(ft)
    
    split = 'test'

    filename = lookup[cocoid]['filename']
    filepath = lookup[cocoid]['filepath']
    imgid = lookup[cocoid]['imgid']
    posTag = 'Original' 

    #first the original captions for this cocoid
    
    for org_cap in val_captions_per_image[cocoid]:
        
        sent = org_cap[0]
        
        sent = get_tokenized_sentence(sent)
        
        sentid = org_cap[1]
        
        #print(sent)
        changeid = 'Original'

        sentids = [sentid]

        im_dict = defaultdict()
        
        im_dict['change_id'] = changeid
        im_dict['cocoid'] = cocoid
        im_dict['filename'] = filename
        im_dict['filepath'] = filepath
        im_dict['imgid'] = imgid
        im_dict['posTag'] = posTag
        im_dict['sentids'] = sentids
        im_dict['split'] = split
        im_dict['targetfoilPair'] = targetfoilPair


        im_sent_dict = defaultdict()
        
        im_sent_dict['imgid'] = imgid
        im_sent_dict['raw'] = sent
        im_sent_dict['sentid'] = sentid
        im_sent_dict['tokens'] = sent.split()

        im_dict['sentences'] = [im_sent_dict]
      
        test_check_dataset['images'].append(im_dict)
        

    c = 1 #foil counter per caption
    
    #then all the foil captions
    for s in sentences:
    
        #print(s)
        foil_sent = s[0]
        pair0 = s[1]
        pair1 = s[2]
        sentid = s[3]

        targetfoilPair = pair0 + '_' + pair1
        
        changeid = str(sentid) + '_' + str(c)
        c += 1
        sentids = [sentid]
        
        posTag = 'noun'
        
        im_dict = defaultdict()
        
        im_dict['change_id'] = changeid
        im_dict['cocoid'] = cocoid
        im_dict['filename'] = filename
        im_dict['filepath'] = filepath
        im_dict['imgid'] = imgid
        im_dict['posTag'] = posTag
        im_dict['sentids'] = sentids
        im_dict['split'] = split
        im_dict['targetfoilPair'] = targetfoilPair


        im_sent_dict = defaultdict()
        
        im_sent_dict['imgid'] = imgid
        im_sent_dict['raw'] = foil_sent
        im_sent_dict['sentid'] = sentid
        im_sent_dict['tokens'] = foil_sent.split()

        im_dict['sentences'] = [im_sent_dict]
      
        test_check_dataset['images'].append(im_dict)

In [ ]:
with open('train_dataset.json', 'w') as f:
    json.dump(train_check_dataset, f)
    
with open('test_dataset.json', 'w') as f:
    json.dump(test_check_dataset, f)

In [35]:
with open("annotations_trainval/instances_train2017.json") as in_file:
    instances_train = json.load(in_file)

In [36]:
with open("annotations_trainval/instances_val2017.json") as in_file:
    instances_val = json.load(in_file)

In [37]:
instances_train.keys()

dict_keys(['info', 'licenses', 'images', 'annotations', 'categories'])

In [38]:
instances_train['annotations'][0]

{'area': 2765.1486500000005,
 'bbox': [199.84, 200.46, 77.71, 70.88],
 'category_id': 58,
 'id': 156,
 'image_id': 558840,
 'iscrowd': 0,
 'segmentation': [[239.97,
   260.24,
   222.04,
   270.49,
   199.84,
   253.41,
   213.5,
   227.79,
   259.62,
   200.46,
   274.13,
   202.17,
   277.55,
   210.71,
   249.37,
   253.41,
   237.41,
   264.51,
   242.54,
   261.95,
   228.87,
   271.34]]}

In [39]:
for inst in instances_train['annotations']:
    if inst['image_id'] == 558840:
        print(inst['category_id'])

58
44
44
47
1
44
50
1
44
67
50


In [40]:
for cat in instances_train['categories']:
    if cat['id']== 67:
        print(cat['supercategory'],cat['name'])

furniture dining table


In [41]:
def get_category_lookup(instances_set):
    
    category_lookup = defaultdict(lambda: defaultdict(str))
    
    for cat in instances_train['categories']:
        
        category_lookup[cat['id']] = {'name':cat['name'], 'supercategory':cat['supercategory']}
    
    return category_lookup

In [42]:
category_lookup = get_category_lookup(instances_train)

In [43]:
def get_image_object_instances(instances_set):
    
    instances = defaultdict(set)
    
    for inst in instances_set['annotations']:
        category_name_supcat = category_lookup[inst['category_id']]
        instances[inst['image_id']].add(category_name_supcat['name'])
        
    return instances

In [44]:
train_object_instances = get_image_object_instances(instances_train)

In [45]:
val_object_instances = get_image_object_instances(instances_val)

In [46]:
train_object_instances[558840]

{'bottle', 'cup', 'dining table', 'hot dog', 'person', 'spoon'}

In [47]:
# def get_multiword_obj(categories):
    
#     multiword_dicts = defaultdict(list)
    
#     for cat in categories:
#         name = cat['name']
        
#         words = name.split()
        
#         if len(words) > 1:
#             #multiword
            
#             first = words[0]
#             second = words[1]
#             concat_without_space = words[0] + words[1]
    
        
#             multiword_dicts[name] = [first, second, concat_without_space]
            
    
#     return multiword_dicts
    

# multiword_dicts = get_multiword_obj(test_cap['categories'])

# multiword_dicts